In [1]:
import string
import os, sys
from pathlib import Path
import urllib
# import urllib2
import datetime
import time
import shutil
from glob import glob
import gzip
import numpy as np
import pandas as pd
import re

import json
import pickle

from collections import defaultdict

In [175]:
data_root = Path('/home/yl986/data/HINT/')
source_root = data_root / 'uniprot_source/release_202401/knowledgebase'
# output_root = data_root / 'outputs_2023'
update_root = data_root / 'update_2024'
output_root = update_root / 'outputs'

### Continue to fill UniProt IDs

**Based on data sources from raw_interactions.txt**
```{python}
target_ids = ['BioGRID',
                'ChEMBL',
                'ComplexPortal',
                'DIP',
                'EMBL',
                'Ensembl',
                'EnsemblGenome',
                'GeneID',
                'PDB',
                'Reactome',
                'RefSeq']
```

**Prior steps**

* Run `parse_source_data.py` to process
    * FASTA files (`uniprot_sprot.fasta.gz`, `uniprot_sprot_varsplic.fasta.gz`, `uniprot_trembl.fasta.gz`) - to extract protein meta information
    * species file `docs/speclist.txt` to extract species taxonomy information
    * secondary-to-primary accession mapping file `docs/sec_ac.txt`
* Run `prepare_dataset.py` to prepare protein interaction data from sources of interest

* Run `create_idmapping.py`: parse `idmapping.dat.gz` file obtained from UniProt FTP site
    * output 1: `target_type_to_uprot.json` dictionary of ID mapping organized by ID type (if an ID is mapped to multiple UniProt IDs, all UniProt IDs will be kept & concatenated by `'|'`)
    * output 2: `prot_gene_info.tsv` descriptions for each UniProt ID columns: `(uprot | UniProtKB-ID | Gene_Name | Gene_ORFName | NCBI_TaxID)`
    * `target_result.json` mapping of target IDs to UniProt when available  
    Format: `"ID_TYPE|SOURCE_ID": "UNIPROT_ID1(|UNIPROT_ID2|UNIPROT_ID3...)"` (Example: '"DIP|DIP-17064N": "Q9TW27"')

    
    
<!-- * Run `map_ids.py`: map target IDs to UniProt IDs
    * Inputs: 
        * `accepted_id_dict.json`: mapping of ID names from original database to supported ID type keywords in `idmapping.dat.gz`
        * `target_type_to_uprot.json` reference for ID mapping organized by ID type
        * `mapping_targets_by_type.json`: target IDs to map organized by ID type
    * Outputs:
        * `target_result.json` mapping of target IDs to UniProt when available  
        
          Format: `"ID_TYPE|SOURCE_ID": "UNIPROT_ID1(|UNIPROT_ID2|UNIPROT_ID3...)"` (Example: '"DIP|DIP-17064N": "Q9TW27"') -->

* Load cache interaction file

In [7]:
raw_interactions1 = pd.read_csv(output_root / 'cache/raw_interactions_filled_partial.txt', sep='\t', dtype={'method': str})
raw_interactions1['method'] = raw_interactions1['method'].str.zfill(4)

/tmp/ipykernel_4078056/2836094877.py:1: DtypeWarning: Columns (2,3,5,6,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_interactions1 = pd.read_csv(output_root / 'cache/raw_interactions_filled_partial.txt', sep='\t', dtype={'method': str})


* Load accepted ID to keyword mapping

In [10]:
# with open(data_root / 'outputs_2023/cache/accepted_id_dict.json', 'r') as f:
#     accept_id_map = json.load(f)

# Global ID dictionary for FTP data - maps different ID types to their UNIPROT DB KEY
# values are valid ID types from uniprot idmapping.dat file
accept_id_map = {'refseq': "RefSeq", 
                'biogrid': "BioGRID", 
                'chembl target': "ChEMBL", 
                'complex': "ComplexPortal", 
                'ddbj/embl/genbank': "EMBL",
                'dip': "DIP",
                'ensembl': "Ensembl", 
                'ensemblgenomes': "EnsemblGenome",
                'entrezgene/locuslink': "GeneID",
                'entrez gene/locuslink': "GeneID",
                'reactome': "Reactome",
                'uniprot/swiss-prot': "UniProtKB-Swiss-Prot", 
                'uniprotkb': "UniProtKB",
                'wwpdb': "PDB"}

In [11]:
# Load FTP mapping result
with open(output_root / 'cache/target_result.json') as f:
    target_map_dict = json.load(f)

In [ ]:
# Load REST-API mapping result (consider to discard this step and only use FTP result)
# df_map2 = pd.read_csv(output_root / 'cache/api_id_map_result.txt', sep='\t', names=['source_id', 'uprot'])
# df_map2 = df_map2.groupby('source_id').agg({'uprot': lambda x: '|'.join(sorted(set(x)))}).reset_index()
# df_map2['source_id'] = df_map2['source_id'].str.replace('RefSeq_Protein', 'RefSeq')
# df_map2['source'] = df_map2['source_id'].apply(lambda x: x.split('|')[0])

# # update target_map_dict
# target_map_dict.update(dict(zip(df_map2['source_id'], df_map2['uprot'])))

In [ ]:
# check remaining IDs to be mapped
# raw_interactions1[raw_interactions1['UniProt_A'].isna() | raw_interactions1['UniProt_B'].isna()]['source'].value_counts()

#### Map source ID to UniProt (multiple mapping exists)

In [13]:
def map_to_uniprot(itype, source_id, accept_id_map, target_map_dict, default_val=np.nan):
    if itype in accept_id_map:
        key = '|'.join([accept_id_map[str(itype)], str(source_id)])
    elif itype in accept_id_map.values():
        key = '|'.join([itype, str(source_id)])
    else:
        return default_val
    return target_map_dict.get(key, default_val)

In [14]:
idx = raw_interactions1['UniProt_A'].isna()
raw_interactions1.loc[idx, 'UniProt_A'] = raw_interactions1.loc[idx].apply(lambda x: map_to_uniprot(x['idtype_A'], x['idA'], accept_id_map, target_map_dict, x['UniProt_A']), axis=1)

idx = raw_interactions1['UniProt_B'].isna()
raw_interactions1.loc[idx, 'UniProt_B'] = raw_interactions1.loc[idx].apply(lambda x: map_to_uniprot(x['idtype_B'], x['idB'], accept_id_map, target_map_dict, x['UniProt_B']), axis=1)

In [15]:
# save cache if necessary
# raw_interactions1.to_csv(output_source / 'cache/raw_interactions_filled_all.txt', sep='\t', index=False)

In [16]:
# proceed with filled entries
raw_ppi_filled = raw_interactions1[raw_interactions1['UniProt_A'].notna() & raw_interactions1['UniProt_B'].notna()].reset_index(drop=True)

In [17]:
# Additional cleaning steps
if raw_ppi_filled['taxa'].str.contains('\.').any():
    raw_ppi_filled['taxa'] = raw_ppi_filled['taxa'].apply(lambda x: str(x).split('.')[0])
if raw_ppi_filled['idA'].str.contains(' ').any():
    raw_ppi_filled['idA'] = raw_ppi_filled['idA'].apply(lambda x: str(x).split(' ')[0].strip())
if raw_ppi_filled['idB'].str.contains(' ').any():
    raw_ppi_filled['idB'] = raw_ppi_filled['idB'].apply(lambda x: str(x).split(' ')[0].strip())

In [18]:
raw_ppi_filled.shape

(5165045, 11)

* save to cache

In [20]:
raw_ppi_filled.to_csv(output_root / 'cache/raw_interactions_filled_all.txt', index=False, sep='\t')

#### Re-format ID-to-UniProt mapping file (one protein ID each row)

In [21]:
df_prot = pd.concat([raw_interactions1[['idtype_A', 'idA', 'taxa', 'UniProt_A']].rename(columns={'idtype_A': 'idtype', 'idA': 'id', 'UniProt_A': 'UniProt'}), 
                     raw_interactions1[['idtype_B', 'idB', 'taxa', 'UniProt_B']].rename(columns={'idtype_B': 'idtype', 'idB': 'id', 'UniProt_B': 'UniProt'})])
df_prot = df_prot.dropna().drop_duplicates().reset_index(drop=True)

In [23]:
df_prot['taxa'] = df_prot['taxa'].astype(str)

In [25]:
df_prot[df_prot['taxa'].str.contains('9606')]['taxa'].unique()

array(['9606', '9606.0'], dtype=object)

In [27]:
df_prot.loc[df_prot['taxa'].str.contains('-')]

,idtype,id,taxa,UniProt
108669,uniprotkb,P31939,-,P31939
282302,uniprotkb,Q9UBE0,-,Q9UBE0


In [28]:
# Manual cleaning
# checked manually: P31939, Q9UBE0 are both human protein (reviewed)
df_prot.loc[df_prot['taxa'].str.contains('-'), 'taxa'] = '9606'
df_prot['taxa'] = df_prot['taxa'].apply(lambda x: str(x).split('.')[0])  # clean up float format taxa ID (result from NaN in the column)

* Split entries with joined UniProt IDs into multiple rows

In [29]:
df_prot['uprot_lst'] = df_prot['UniProt'].str.split('|')
df_prot = df_prot.explode('uprot_lst').drop('UniProt', axis=1).drop_duplicates().rename(columns={'uprot_lst': 'UniProt'}).reset_index(drop=True)

In [33]:
# sanity check
# df_prot.describe()

In [32]:
cleanup_idx = df_prot['UniProt'].str.contains(' ')
df_prot.loc[cleanup_idx, 'id'] = df_prot.loc[cleanup_idx, 'id'].apply(lambda x: x.strip().split(' ')[0])
df_prot.loc[cleanup_idx, 'UniProt'] = df_prot.loc[cleanup_idx, 'UniProt'].apply(lambda x: x.strip().split(' ')[0])

In [36]:
# save cache file
df_prot.to_csv(output_root / 'cache/id_to_uniprot.txt', sep='\t', index=False)

#### Secondary accession file

In [34]:
# df_prot = pd.read_csv(output_root / 'cache/id_to_uniprot.txt', sep='\t')

In [38]:
df_prot['UniProt'].nunique()

218601

In [39]:
sec_merged = pd.read_csv(source_root / 'docs/cache/sec_acc_parsed.txt', sep='\t')

In [40]:
sec_merged.head()

,secondary,primary
0,A0A011PKA5,C7RW80
1,A0A011PPS3,C7RW54
2,A0A011Q4P3,C7RWC7
3,A0A011QYZ9,C7RW92
4,A0A016SR66,A0A0D6L478


In [41]:
sec_uprots = df_prot[df_prot['UniProt'].isin(sec_merged['secondary'])]['UniProt'].unique()
len(sec_uprots)

3085

In [42]:
df_prot = df_prot.merge(sec_merged.rename(columns={'secondary': 'UniProt'}), how='left').drop_duplicates()
df_prot.loc[df_prot['primary'].isna(), 'primary'] = df_prot.loc[df_prot['primary'].isna(), 'UniProt']

In [43]:
df_prot['primary'].nunique()

216993

In [44]:
df_prot.dtypes

idtype     object
id         object
taxa       object
UniProt    object
primary    object
dtype: object

In [45]:
df_prot['primary_short'] = df_prot['primary'].apply(lambda x: x.split('-')[0])
df_prot['primary_short'].nunique()

204145

* save source ID to UniProt mapping file

In [49]:
df_prot.head()

,idtype,id,taxa,UniProt,primary,primary_short
0,dip,DIP-445N,10090,P46414,P46414,P46414
1,dip,DIP-617N,9606,P01730,P01730,P01730
2,dip,DIP-1025N,83333,P00968,P00968,P00968
3,dip,DIP-19N,7227,P07713,P07713,P07713
4,dip,DIP-25N,4932,P09798,P09798,P09798


In [47]:
df_prot.to_csv(output_root / 'cache/id_to_uniprot.txt', sep='\t', index=False)

In [48]:
# df_prot = pd.read_csv(output_root / 'cache/id_to_uniprot.txt', sep='\t', dtype={'taxa': str})

### Merge with protein description data

**Preparations**

* Generate `uniprot_descriptions.txt` (description for selected protein) from protein meta files (`query_uprot_desc.py`)
  * Input: 
    * `id_to_uniprot.txt`
    * protein meta files under $UNIPROT_KNOWLEDGEBASE/complete/meta

**Either run the following chunks or the python script**


In [ ]:
def extract_uprot_meta_info(df_prot, source_path, out_path, pid_col='UniProt', meta_pid_col='UniProt', 
                            meta_prefix=['sprot', 'sprot_varsplic', 'trembl'], chunksize=1e6, 
                            output_name='uniprot_descriptions.txt'):
    """
    Extract description for selected protein IDs
    """
    if isinstance(source_path, str):
        source_path = Path(source_path)

    if isinstance(out_path, str):
        out_path = Path(out_path)
    
    if not out_path.exists():
        out_path.mkdir(parents=True)

    prot_remain = set(df_prot[pid_col])
    print('# UniProt IDs:', len(prot_remain))

    header = True
    mode = 'w'
    for prefix in meta_prefix:
        fname = f'{prefix}_meta.txt'
        if not (source_path / fname).exists():
            print('{} not found!'.format(str(source_path / fname)))
            continue

        chunks = pd.read_csv(source_path / fname, sep='\t', chunksize=chunksize)
        for df_meta in chunks:
            records = df_meta[df_meta[meta_pid_col].isin(prot_remain)]
            records.to_csv(out_path / output_name, mode=mode, header=header, sep='\t', index=False)
            prot_remain = prot_remain - set(records[meta_pid_col])
            print('# UniProt IDs remain:', len(prot_remain))
            if len(prot_remain) == 0:
                break
            header = False  # write header only the first time
            mode = 'a'

In [ ]:
# Uncomment to run

# prot_meta_root = Path('/home/yl986/data/HINT/uniprot_source/release_202401/knowledgebase/complete/meta')
# df_prot = pd.read_csv(data_root / 'id_to_uniprot.txt', sep='\t')
# output_cache_root = output_root / 'cache'

# extract_uprot_meta_info(df_prot, prot_meta_root, out_path=output_cache_root, pid_col='primary_ac_short')

* Continue with processed uniprot description file

In [50]:
df_desc = pd.read_csv(output_root / 'cache/uniprot_descriptions.txt', sep='\t')

In [51]:
df_desc.head()

,tag,UniProt,name,length,species,taxa,gene_name,description
0,sp,P32234,128UP_DROME,368,Drosophila melanogaster,7227,128up,GTP-binding protein 128up
1,sp,Q8GBW6,12S_PROFR,524,Propionibacterium freudenreichii subsp. shermanii,1752,NaN,Methylmalonyl-CoA carboxyltransferase 12S subunit
2,sp,P81928,140U_DROME,261,Drosophila melanogaster,7227,140up,RPII140-upstream gene protein
3,sp,P48347,14310_ARATH,254,Arabidopsis thaliana,3702,GRF10,14-3-3-like protein GF14 epsilon
4,sp,Q9S9Z8,14311_ARATH,252,Arabidopsis thaliana,3702,GRF11,14-3-3-like protein GF14 omicron


In [55]:
df_desc['taxa'].dtype

dtype('int64')

#### Curate species information

**Note**

Parsed species information from `parse_source_data.py` needs further processing to merge "redundant" taxonomy IDs

In [52]:
df_species = pd.read_csv(source_root / 'docs/cache/species_parsed.txt', sep='\t')

In [53]:
df_species['taxa'] = df_species['taxa'].astype(str)

In [54]:
df_species['taxa'].isna().any()

False

In [56]:
# Rule-based curation (consider to revise later)
df_species['scientific_name_short'] = df_species['scientific_name'].apply(lambda x: x.split('(')[0].strip())
ecoli_idx = df_species['scientific_name'].str.startswith('Escherichia coli')  # manual rule for E Coli
df_species.loc[ecoli_idx, 'scientific_name_short'] = 'Escherichia coli'

rice_idx = df_species['common_name'] == 'Rice' # manual rule for Rice
df_species.loc[rice_idx, 'scientific_name_short'] = 'Oryza sativa'

In [57]:
# generate aggregated taxa ID mapping dict

df_spe_agg = df_species.astype({'taxa': int}).groupby('scientific_name_short').agg({'taxa': lambda x: sorted(list(x))}).reset_index()
df_spe_agg['n_taxa'] = df_spe_agg['taxa'].apply(len)
df_multi = df_spe_agg.query('n_taxa > 1').reset_index(drop=True)
df_multi['taxa_unified'] = df_multi['taxa'].apply(lambda x: x[0])  # map to the smallest ID value within the same "species"
df_multi = df_multi.explode('taxa').reset_index(drop=True)

taxa_map_dict = dict(zip(df_multi['taxa'].astype(str), df_multi['taxa_unified'].astype(str)))

In [58]:
len(taxa_map_dict)

4496

* save taxanomy mapping file

In [59]:
with open(output_root / 'cache/taxa_map_dict.json', 'w') as f:
    json.dump(taxa_map_dict, f, indent=2)

#### Continue to clean up protein mapping & description

In [60]:
df_prot['taxa'].dtype

dtype('O')

In [61]:
df_prot['taxa'].nunique()

5107

In [62]:
# revise taxa ID first

df_prot['taxa_rev'] = df_prot['taxa'].astype(str).apply(lambda x: taxa_map_dict[str(x)] if x in taxa_map_dict else x)
print(df_prot['taxa_rev'].nunique())

df_desc['taxa_rev'] = df_desc['taxa'].astype(str).apply(lambda x: taxa_map_dict[str(x)] if x in taxa_map_dict else x)
print(df_desc['taxa_rev'].nunique())

4395
4131


In [63]:
# Merge by UniProt AND unified taxa

pid_col = 'primary_short'

df_prot_info = df_prot[['idtype', 'id', 'taxa_rev', 'UniProt', 'primary_short']].rename(columns={'UniProt': 'UniProt_orig'}).\
                merge(df_desc, left_on=['primary_short', 'taxa_rev'], right_on=['UniProt', 'taxa_rev'], how='left').drop(columns=['UniProt', 'taxa']).\
                rename(columns={'taxa_rev': 'taxa'}).drop_duplicates().reset_index(drop=True)

In [64]:
df_prot_info['taxa'].nunique()

4395

In [65]:
df_prot_info.shape

(376182, 11)

In [66]:
df_prot_info_fil = df_prot_info.dropna(subset='name').drop_duplicates().reset_index(drop=True)
df_prot_info_fil['taxa'].nunique()

4015

In [67]:
df_prot_info_fil.shape

(369964, 11)

In [335]:
# Goal: filtering to keep only mappings with consistent taxonmy
# taxa_x: taxonomy ID in source (reference)
# taxa_y: taxonomy ID from id mapping file generated by `merge` operation
#         (may not be accurate because some secondary IDs map to multiple primary IDs regardless of taxonomy information)
# df_prot_info['taxa_x'] = df_prot_info['taxa_x'].apply(lambda x: taxa_map_dict[str(x)] if x in taxa_map_dict else x)
# df_prot_info['taxa_y'] = df_prot_info['taxa_y'].apply(lambda x: taxa_map_dict[str(x)] if x in taxa_map_dict else x)

In [336]:
# remove inconsistent taxa entries
# df_prot_info_fil = df_prot_info.query('taxa_x == taxa_y')
# df_prot_info_fil = df_prot_info_fil.rename(columns={'taxa_x': 'taxa'}).drop('taxa_y', axis=1)

# df_prot_info_fil['primary_short'].nunique()

In [68]:
df_prot_info_fil['is_reviewed'] = (df_prot_info_fil['tag'] == 'sp').astype(int)

In [69]:
df_prot_info_fil.head()

,idtype,id,taxa,UniProt_orig,primary_short,tag,name,length,species,gene_name,description,is_reviewed
0,dip,DIP-445N,10090,P46414,P46414,sp,CDN1B_MOUSE,197.0,Mus musculus,Cdkn1b,Cyclin-dependent kinase inhibitor 1B,1
1,dip,DIP-617N,9606,P01730,P01730,sp,CD4_HUMAN,458.0,Homo sapiens,CD4,T-cell surface glycoprotein CD4,1
2,dip,DIP-1025N,562,P00968,P00968,sp,CARB_ECOLI,1073.0,Escherichia coli (strain K12),carB,Carbamoyl phosphate synthase large chain,1
3,dip,DIP-19N,7227,P07713,P07713,sp,DECA_DROME,588.0,Drosophila melanogaster,dpp,Protein decapentaplegic,1
4,dip,DIP-25N,4932,P09798,P09798,sp,CDC16_YEAST,840.0,Saccharomyces cerevisiae (strain ATCC 204508 /...,CDC16,Anaphase-promoting complex subunit CDC16,1


In [73]:
# df_prot_info_fil['info_tup'] = df_prot_info_fil.apply(lambda x: (x['UniProt'], x['is_reviewed']), axis=1)

In [74]:
df_prot_info_fil[df_prot_info_fil['gene_name'].isna()]['UniProt_orig'].nunique()

8111

In [75]:
df_prot_info_fil[~df_prot_info_fil['UniProt_orig'].isin(sec_merged['secondary']) & 
                 (df_prot_info_fil['UniProt_orig'] != df_prot_info_fil['primary_short']) & 
                 ~df_prot_info_fil['UniProt_orig'].str.contains('-')].drop_duplicates('UniProt_orig')

,idtype,id,taxa,UniProt_orig,primary_short,tag,name,length,species,gene_name,description,is_reviewed


In [76]:
na_gene_idx = df_prot_info_fil['gene_name'].isna()
is_sec = df_prot_info_fil['UniProt_orig'].isin(sec_merged['secondary'])
is_isoform = df_prot_info_fil['UniProt_orig'].str.contains('-')

In [78]:
# sanity check
# df_prot_info_fil[na_gene_idx & is_sec]

In [80]:
# sanity check
# df_prot_info_fil[na_gene_idx & is_isoform]

#### Reduce to best UniProt IDs by gene

##### Fill empty gene name with UniProt ID

From sanity check:
* UniProt isoforms without gene info are all viral proteins (maybe keep orginal UniProt ID with suffix later) -- use prefix for now
* secondary accessions without gene info: use primary accession
* others: use primary accessions

**Note**
For UniProt IDs with suffix ('-PRO', isoform...), now we fill empty gene name with only the prefix (may want to use the full UniProt name in final display)

In [81]:
df_prot_info_fil['gene_name_fil'] = df_prot_info_fil['gene_name']
df_prot_info_fil.loc[na_gene_idx, 'gene_name_fil'] = df_prot_info_fil.loc[na_gene_idx, 'primary_short']
# df_prot_info_fil.loc[na_gene_idx & is_isoform, 'gene_name_fil'] = df_prot_info_fil.loc[na_gene_idx, 'UniProt_orig']

##### Find best UniProt ID for each gene

In [82]:
df_prot_info_fil['gene_name_fil'].nunique()

136874

In [83]:
df_prot_info_fil['name'].isna().any()

False

In [84]:
df_prot_info_fil['id'].nunique()

286075

In [332]:
# df_prot_info_fil = df_prot_info_fil.sort_values(['gene_name_fil', 'taxa', 'is_reviewed'], ascending=[True, True, False])

In [154]:
# best_by_id = df_prot_info_fil.sort_values(['id','taxa', 'is_reviewed', 'gene_name_fil'], ascending=[True, True, False, True]).drop_duplicates(['id', 'taxa'], keep='first')

In [85]:
best_by_gene = df_prot_info_fil.sort_values(['gene_name', 'taxa', 'is_reviewed'], ascending=[True, True, False]).drop_duplicates(['gene_name_fil', 'taxa'], keep='first')

In [86]:
best_by_gene['is_reviewed'].value_counts()

1    79995
0    77171
Name: is_reviewed, dtype: int64

In [87]:
best_by_gene[['taxa', 'gene_name_fil', 'primary_short']].astype({'taxa': str}).describe()

,taxa,gene_name_fil,primary_short
count,157166,157166,157166
unique,4015,136874,157166
top,9606,HA,P03775
freq,24386,110,1


In [91]:
gene2best_uprot_bytaxa = defaultdict(dict)

for cur_tid in best_by_gene['taxa'].unique():
    df_taxa = best_by_gene.query('taxa == @cur_tid')
    gene2best_uprot_bytaxa[cur_tid] = dict(zip(df_taxa['gene_name_fil'], df_taxa['primary_short']))

In [92]:
with open(output_root / 'cache/gene2best_uprot_bytaxa.json', 'w') as f:
    json.dump(gene2best_uprot_bytaxa, f, indent=2)

In [93]:
len(set(taxa_map_dict.values()))

1263

In [120]:
df_prot_info_fil['length'] = df_prot_info_fil['length'].astype(int)

In [94]:
df_prot_info_fil['best_uprot'] = df_prot_info_fil.apply(lambda x: gene2best_uprot_bytaxa[x['taxa']][x['gene_name_fil']], axis=1)

In [122]:
df_prot_info_fil.drop_duplicates().to_csv(output_root / 'cache/mapped_to_best_uprot.txt', sep='\t', index=False)

### Apply best UniProt mapping to interactions

In [ ]:
# Load supporting documents from cache (if necessary)

# taxa ID mapping
with open(output_root / 'cache/taxa_map_dict.json', 'r') as f:
    taxa_map_dict = json.load(f)

# best UniProt ID mapping (by gene and taxa)
with open(output_root / 'cache/gene2best_uprot_bytaxa.json', 'r') as f:
    gene2best_uprot_bytaxa = json.load(f)

# species information
df_species = pd.read_csv(source_root / 'docs/cache/species_parsed.txt', sep='\t')

# protein descriptions
df_desc = pd.read_csv(output_root / 'cache/uniprot_descriptions.txt', sep='\t')

# ID mapping (to best UniProt)
df_prot_info_fil = pd.read_csv(output_root / 'cache/mapped_to_best_uprot.txt', sep='\t', 
                            dtype={'id': str, 'taxa': str, 'gene_name': str, 'gene_name_fil': str})

# Raw interactions with filled UniProt (orig)
raw_ppi_filled = pd.read_csv(output_root / 'cache/raw_interactions_filled_all.txt', sep='\t', dtype={'method': str, 'taxa': str, 'publ': str})

In [319]:
df_prot_info_fil = pd.read_csv(output_root / 'cache/mapped_to_best_uprot.txt', sep='\t', 
                            dtype={'id': str, 'taxa': str, 'gene_name': str, 'gene_name_fil': str})

In [301]:
dtype_dict = {'method': str, 'idA': str, 'idB': str, 'taxa': str, 'publ': str}

raw_ppi_filled = pd.read_csv(output_root / 'cache/raw_interactions_filled_all.txt', sep='\t', dtype=dtype_dict)

* drop interactions with NA taxonamy ID

In [107]:
raw_ppi_filled = raw_ppi_filled.dropna(subset=['taxa']).reset_index(drop=True)

In [108]:
raw_ppi_filled['taxa'].isna().any()

False

In [109]:
df_prot_info_fil['best_uprot'].nunique()

157166

In [111]:
# Additional cleaning steps
if raw_ppi_filled['taxa'].str.contains('\.').any():
    raw_ppi_filled['taxa'] = raw_ppi_filled['taxa'].apply(lambda x: str(x).split('.')[0])
if raw_ppi_filled['idA'].str.contains(' ').any():
    raw_ppi_filled['idA'] = raw_ppi_filled['idA'].apply(lambda x: str(x).split(' ')[0].strip())
if raw_ppi_filled['idB'].str.contains(' ').any():
    raw_ppi_filled['idB'] = raw_ppi_filled['idB'].apply(lambda x: str(x).split(' ')[0].strip())

In [112]:
raw_ppi_filled.shape

(5159751, 11)

In [295]:
# raw_ppi_filled.drop('row_number', axis=1).describe()

In [305]:
# check secondary accessions
# is_sec = df_prot_info_fil['UniProt_orig'].isin(sec_merged['secondary'])
# df_prot_info_fil[is_sec]

#### Revise for isoform

In [113]:
is_isoform = df_prot_info_fil['UniProt_orig'].str.contains('-')
is_sec = df_prot_info_fil['UniProt_orig'].isin(sec_merged['secondary'])

df_prot_info_fil['best_uprot_full'] = df_prot_info_fil['best_uprot']
df_prot_info_fil.loc[is_isoform, 'best_uprot_full'] = df_prot_info_fil.loc[is_isoform, 'UniProt_orig']

# use original UniProt ID when given in source
uprot_source_idx = df_prot_info_fil['idtype'].str.contains('uniprot') & (df_prot_info_fil['UniProt_orig'].str.contains('-'))
df_prot_info_fil.loc[uprot_source_idx & ~is_sec, 'best_uprot_full'] = df_prot_info_fil.loc[uprot_source_idx, 'UniProt_orig']

In [114]:
df_prot_info_fil.query('id == "NP_536350.2"')

,idtype,id,taxa,UniProt_orig,primary_short,tag,name,length,species,gene_name,description,is_reviewed,gene_name_fil,best_uprot,best_uprot_full
233794,refseq,NP_536350.2,9606,P63092,P63092,sp,GNAS2_HUMAN,394.0,Homo sapiens,GNAS,Guanine nucleotide-binding protein G(s) subuni...,1,GNAS,P63092,P63092
233795,refseq,NP_536350.2,9606,Q5JWF2-1,Q5JWF2,sp,GNAS1_HUMAN,1037.0,Homo sapiens,GNAS,Guanine nucleotide-binding protein G(s) subuni...,1,GNAS,P63092,Q5JWF2-1


* reduce to unique ID mapping (some raw IDs map to multiple gene names --> multiple UniProt IDs)

In [115]:
# best Uniprot same as original UniProt ID or not
df_prot_info_fil['same_uprot'] = (df_prot_info_fil['primary_short'] == df_prot_info_fil['best_uprot_full']).astype(int)
df_prot_info_fil['is_canonical'] = ~df_prot_info_fil['best_uprot_full'].str.contains('-')

In [123]:
to_best_uprot = df_prot_info_fil.sort_values(['id', 'taxa', 'is_reviewed', 'same_uprot', 'is_canonical', 'best_uprot_full'],
                                          ascending=[True, True, False, False, False, False]).drop_duplicates(['idtype', 'id', 'taxa'], keep='first').reset_index(drop=True)

* save cache file

In [124]:
to_best_uprot.to_csv(output_root / 'cache/id_to_best_uprot_uniq.txt', index=False, sep='\t')

In [125]:
all_taxa = to_best_uprot['taxa'].unique()

uniq_id_mapping_by_taxa = dict()

for taxa in all_taxa:
    df_cur = to_best_uprot.query('taxa == @taxa')
    uniq_id_mapping_by_taxa[taxa] = dict(zip(df_cur['id'], df_cur['best_uprot']))

#### Revise taxonomy ID in raw interaction data

In [126]:
print(raw_ppi_filled[raw_ppi_filled['taxa'] == '-']['row_number'].nunique())
raw_ppi_filled = raw_ppi_filled.query('taxa != "-"').reset_index(drop=True)
raw_ppi_filled['taxa_rev'] = raw_ppi_filled['taxa'].apply(lambda x: int(taxa_map_dict.get(str(x), x))).astype(str)

1


In [127]:
to_best_uprot.query('best_uprot_full == "X2JAU8"').iloc[0]['taxa']

'7227'

In [128]:
raw_ppi_filled.query('row_number == 1533613').iloc[0]['taxa_rev']

'7227'

In [129]:
raw_ppi_filled['taxa_rev'].dtype

dtype('O')

In [130]:
cols = ['idtype', 'id', 'taxa', 'gene_name', 'name', 'best_uprot_full']
ppi_merge = raw_ppi_filled.drop('taxa', axis=1).rename(columns={'taxa_rev': 'taxa'}).\
                merge(to_best_uprot[cols], left_on=['idtype_A', 'idA', 'taxa'], right_on=['idtype', 'id', 'taxa'], how='left').\
                rename(columns={'best_uprot_full': 'best_uprotA', 'gene_name': 'gene_name_A', 'name': 'entry_name_A'}).drop(columns=['idtype', 'id'])
ppi_merge = ppi_merge.merge(to_best_uprot[cols], left_on=['idtype_B', 'idB', 'taxa'], right_on=['idtype', 'id', 'taxa'], how='left').\
                rename(columns={'best_uprot_full': 'best_uprotB', 'gene_name': 'gene_name_B', 'name': 'entry_name_B'}).drop(columns=['idtype', 'id'])

In [131]:
ppi_merge[ppi_merge['best_uprotA'].notna() & ppi_merge['best_uprotB'].notna()].drop_duplicates(['best_uprotA', 'best_uprotB']).shape

(2412708, 17)

### Apply filtering conditions

In [ ]:
# dtype_dict = {'method': str, 'idA': str, 'idB': str, 'gene_name_A': str, 'gene_name_B': str}
# ppi_merge = pd.read_csv(output_root / 'cache/raw_interactome_draft.txt', sep='\t', dtype=dtype_dict)
# print(ppi_merge.shape[0])

In [132]:
ppi_merge.shape

(5159749, 17)

In [133]:
print("Missing raw ID:", ppi_merge[ppi_merge['idA'].isna() | ppi_merge['idB'].isna()].shape[0])
ppi_merge = ppi_merge[ppi_merge['idA'].notna() & ppi_merge['idB'].notna()].reset_index(drop=True)
ppi_merge.shape

Missing raw ID: 0


(5159749, 17)

In [134]:
ppi_merge.to_csv(output_root / 'cache/raw_interactome_draft.txt', index=False, sep='\t')

In [135]:
# sanity check for missing taxonomy ID
ppi_merge['taxa'].isna().any()

False

#### Proceed with interactions having complete UniProt information

In [136]:
# interactions with complete UniProt information
ppi_complete = ppi_merge[ppi_merge['best_uprotA'].notna() & ppi_merge['best_uprotB'].notna()].reset_index(drop=True)
print(ppi_complete.shape[0])

4795949


In [137]:
# Unique interaction ID (sorted by UniProt ID)
ppi_complete['ppi'] = ppi_complete.apply(lambda x: ':'.join(sorted([x['best_uprotA'], x['best_uprotB']])), axis=1)

In [138]:
ppi_complete['ppi'].nunique()

2082317

* Cleanup evidence code (remove punctuations)

In [139]:
pat = re.compile("[" + re.escape(string.punctuation) + "]")

In [140]:
# sanity check
ppi_complete['method'].str.contains(pat).any()

True

In [141]:
ppi_complete['method'] = ppi_complete['method'].apply(lambda x: re.sub(pat, '', x))

In [142]:
# some contains space and description
ppi_complete[ppi_complete['method'].str.len() > 4]['method'].unique()

array(['0055 fluore', '0012 biolumine'], dtype=object)

In [143]:
ppi_complete.loc[ppi_complete['method'].str.len() > 4, 'method'] = ppi_complete.loc[ppi_complete['method'].str.len() > 4, 'method'].apply(lambda x: x.split(' ')[0])

* Cleanup publication ID

In [152]:
ppi_complete['publ'] = ppi_complete['publ'].astype(str)

In [144]:
ppi_complete['publ'].str.contains(':').any()

True

In [154]:
ppi_complete[ppi_complete['publ'].isna()]

,source,row_number,idA,idB,method,publ,idtype_A,idtype_B,UniProt_A,UniProt_B,taxa,gene_name_A,entry_name_A,best_uprotA,gene_name_B,entry_name_B,best_uprotB,ppi


In [153]:
# additional cleaning for some entries (from DIP, e.g. rtd:11250202, these are pubmed ID)
ppi_complete.loc[ppi_complete['publ'].str.contains(':'), 'publ'] = ppi_complete.loc[ppi_complete['publ'].str.contains(':'), 'publ'].apply(lambda x: x.split(':')[1])

#### Unique interactions

In [155]:
ppi_uniq = ppi_complete.drop_duplicates(['source', 'ppi', 'taxa', 'publ', 'method']).reset_index(drop=True)
ppi_uniq.shape

(3999239, 18)

In [156]:
# sanity check: any interactions with multiple taxa ID?
ppi_uniq.groupby('ppi')['taxa'].nunique().max()

1

In [157]:
ppi_uniq['ppi'].nunique()

2082317

In [158]:
ppi_uniq['pmid_method'] = ppi_uniq.apply(lambda x: '{}:{}'.format(x['publ'], x['method']), axis=1)

In [160]:
ppi_uniq.to_csv(output_root / 'cache/raw_interactome_merge_uniq.txt', index=False, sep='\t')

In [161]:
ppi_uniq.shape

(3999239, 19)

### Prepare for output

In [162]:
ppi_agg = ppi_uniq.groupby('ppi').agg({'source': set, 'pmid_method': set}).reset_index()
ppi_agg['source'] = ppi_agg['source'].apply(lambda x: '|'.join(sorted(x)))
ppi_agg['pmid_method'] = ppi_agg['pmid_method'].apply(lambda x: '|'.join(sorted(x)))

In [163]:
ppi_agg.head()

,ppi,source,pmid_method
0,A0A014C6J9:A0A014C6J9,PDB,29438671:0114
1,A0A021WW32:A0A021WW32,BioGrid,27996020:0004
2,A0A021WW32:A1Z8S6,BioGrid,25588834:0004
3,A0A021WW32:D6W4X5,IntAct,37061542:0397|37061542:1112
4,A0A021WW32:P05205,BioGrid,24990964:0004


In [164]:
ppi_agg = ppi_agg.merge(ppi_uniq[['ppi', 'taxa']].drop_duplicates()).drop_duplicates().reset_index(drop=True)
ppi_agg.shape

(2082317, 4)

In [166]:
ppi_agg['UniProt_A'], ppi_agg['UniProt_B'] = zip(*ppi_agg['ppi'].apply(lambda x: x.split(':')))

In [167]:
# raw interactome data
cols = ['source', 'UniProt_A', 'UniProt_B', 'taxa', 'pmid_method']
ppi_agg[cols].sort_values(['UniProt_A', 'UniProt_B']).to_csv(output_root / 'raw_interactome.txt', index=False, sep='\t')

#### Assign interaction type and quality

##### Assign binary / co-complex by publication and evidence code

In [176]:
ev_ref = pd.read_csv(update_root / 'data/evidence_code_20231120.txt', sep='\t', names=['code', 'name', 'group'], dtype={'code': str})
ev_ref['code'] = ev_ref['code'].str.zfill(4)

* Generate evidence code summary file
--- number of interactions per publication per evidence code

In [177]:
binary_idx = ppi_uniq['method'].isin(ev_ref.query('group == "binary"')['code'])
cocomp_idx = ppi_uniq['method'].isin(ev_ref.query('group == "co-complex"')['code'])

ppi_uniq.loc[binary_idx, 'type'] = 'binary'
ppi_uniq.loc[cocomp_idx, 'type'] = 'co-complex'

for grp in ['binary', 'co-complex']:
    ppi_grp = ppi_uniq.query('type == @grp')
    pub_stats = ppi_grp.groupby(['pmid_method', 'taxa'])['ppi'].nunique().sort_values(ascending=False).reset_index()
    pub_stats['publ'], pub_stats['method'] = zip(*pub_stats['pmid_method'].apply(lambda x: [x.split(':')[-2], x.split(':')[-1]]))
    pub_stats.to_csv(output_root / 'cache/publication_count_{}_with_evi_code.txt'.format(re.sub('-', '', grp)), sep='\t', index=False)

* some evidence code can be either binary or co-complex - evaluate by publication

In [179]:
binary_pub_stats = pd.read_csv(output_root / 'cache/publication_count_binary_with_evi_code.txt', sep='\t', dtype={'method': str})

In [180]:
# evidence code for both binary and co-complex
ev_code_both = [str(s).zfill(4) for s in [6,7,19,59,61,75,96]]
max_ppi = 25

to_cocomp = binary_pub_stats['method'].isin(ev_code_both) & (binary_pub_stats['ppi'] > max_ppi)
to_cocomp_publ = binary_pub_stats[to_cocomp]['pmid_method'].drop_duplicates().tolist()

**Curate publications**

* Binary: (1 OR 2) AND 3
  1. evidence code is binary
  2. evidence code = "0492" or "0493" for HPRD source
  3. publication NOT in blacklist and NOT forced to co-complex

* Co-complex: 1 AND 2
  1. evidence code is co-complex or publication forced as co-complex
  2. publication NOT in blacklist

In [184]:
flist = list((update_root / 'data/bouncer').glob('*.txt'))
publication_bouncer = [line.strip().split('\t') for f in flist for line in open(f).read().strip().split('\n')]

In [185]:
# forced_binary = set()
# pub_blacklist = set()
# forced_cocomp = set()
# general_forced_hq = set()
key_map = {'1': 'forced_binary', '-1': 'pub_blacklist', '*': 'pub_blacklist', '0': 'forced_cocomp', 
           '2': 'general_forced_hq', '-2': 'binary_blacklist_forced_cocomplex'}

special_cases = defaultdict(set)
for pub, _, status in publication_bouncer:
    status = status.strip()
    if status in key_map:
        special_cases[key_map[status]].add(pub.strip())
    elif '*' in status:
        special_cases['pub_blacklist'].add(pub.strip())

In [186]:
def is_binary_pub(pub_id, ev_code, source, exclude_list, binary_methods):
    # exclude_pub = special_cases['pub_blacklist'] + special_cases['binary_blacklist_forced_cocomplex']
    ev_code = str(ev_code).zfill(4)
    if pub_id not in exclude_list:
        if ev_code in binary_methods:
            return True
        if ev_code in ['0492', '0493'] and source.upper() == 'HPRD':
            return True
    return False

def is_cocomp_pub(pub_id, ev_code, blacklist, bin_forced_cocomp, cocomp_methods):
    if pub_id not in blacklist:
        if ev_code in cocomp_methods or pub_id in bin_forced_cocomp:
            return True
    return False

In [187]:
binary_code = ev_ref.query('group == "binary"')['code'].tolist()
cocomp_code = ev_ref.query('group == "co-complex"')['code'].tolist()

binary_exclude = list(special_cases['pub_blacklist']) + list(special_cases['binary_blacklist_forced_cocomplex'])
ppi_uniq['is_binary_pub'] = ppi_uniq.apply(lambda x: is_binary_pub(x['publ'], x['method'], x['source'], binary_exclude, binary_code), axis=1)
ppi_uniq['is_cocomp_pub'] = ppi_uniq.apply(lambda x: is_cocomp_pub(x['publ'], x['method'], special_cases['pub_blacklist'], 
                                                                   special_cases['binary_blacklist_forced_cocomplex'], cocomp_code), axis=1)

##### High-throughput (HT) vs literature-curated (LC)

**Proceed with only binary or co-complex interactions**

* high-throughput
  * binary
    * **binary** publication AND publication in `forced_binary`
  * co-complex
    * **co-complex** publication AND (publication in `forced_cocomp` OR publication selected as cocomplex by cutoff)
* literature curated otherwise

In [189]:
def assign_quality(record, cocomp_pub_by_cutoff, forced_binary, forced_cocomp):
    if record['is_binary_pub']:
        if record['publ'] in forced_binary:
            return 'HT'
        return 'LC'
    
    if record['is_cocomp_pub']:
        if record['publ'] in forced_cocomp or record['publ'] in cocomp_pub_by_cutoff:
            return 'HT'
    
        return 'LC'
    
    return np.nan

In [190]:
# ppi_uniq['quality'] = ppi_uniq.apply(lambda x: assign_quality(x, to_cocomp_publ, special_cases['forced_binary'], special_cases['forced_cocomp']), axis=1)
# ppi_uniq.groupby(['type', 'quality'])['ppi'].nunique()

In [191]:
# proceed with binary / co-complex interactions
ppi_uniq_bincocomp = ppi_uniq[ppi_uniq['is_binary_pub'] | ppi_uniq['is_cocomp_pub']].reset_index(drop=True)
ppi_uniq_bincocomp['ppi'].nunique()

1367310

In [192]:
ppi_uniq_bincocomp['quality'] = ppi_uniq_bincocomp.apply(lambda x: assign_quality(x, to_cocomp_publ, special_cases['forced_binary'], special_cases['forced_cocomp']), axis=1)
ppi_uniq_bincocomp.groupby(['type', 'quality'])['ppi'].nunique()

type        quality
binary      HT         243656
            LC         620592
co-complex  HT         194593
            LC         644359
Name: ppi, dtype: int64

In [240]:
ppi_uniq_bincocomp[ppi_uniq_bincocomp['method_type'].isna() & ppi_uniq_bincocomp['is_binary_pub']]['source'].unique()

array(['HPRD'], dtype=object)

In [238]:
ppi_uniq_bincocomp[ppi_uniq_bincocomp['method_type'].isna()].query('is_cocomp_pub')['publ'].isin(special_cases['binary_blacklist_forced_cocomplex']).all()

True

In [193]:
ppi_uniq_bincocomp['pmid:method:quality'] = ppi_uniq_bincocomp.apply(lambda x: ':'.join([x['publ'], x['method'], x['quality']]), axis=1)

##### High-quality

If an interaction meet any of the following conditions:

* Any publication is binary and is in `forced_binary`
* Any publication co-complex and is in `forced_cocomp` + `forced_cocomp_by_cutoff`
* more than one publication NOT in the blacklist
* source is PDB

In [194]:
def is_high_quality(record, forced_binary, forced_cocomp_all):
    """
    Whether or not a single record (with one publication+method) is high-quality itself
    """
    if record['is_binary_pub'] and record['publ'] in forced_binary:
        return True
    if record['is_cocomp_pub'] and record['publ'] in forced_cocomp_all:
        return True
    if record['source'].upper() == 'PDB':
        return True
    return False

**[revision 2024-06-02] assign quality by interaction type**

In [195]:
def is_high_quality_binary(record, forced_binary):
    """
    For binary interactions:
        Whether or not a single record (with one publication+method) is high-quality itself
    """
    if record['is_binary_pub'] and record['publ'] in forced_binary:
        return True
    if record['source'].upper() == 'PDB':
        return True
    return False

def is_high_quality_cocomp(record, forced_cocomp_all, forced_cocomp_with_ev):
    """
    For co-complex interactions:
        Whether or not a single record (with one publication+method) is high-quality itself
    """
    if record['is_cocomp_pub'] and record['publ'] in forced_cocomp_all:
        return True
    if record['source'].upper() == 'PDB':
        return True
    if record['is_cocomp_pub'] and record['pmid_method'] in forced_cocomp_with_ev:
        return True
    return False

In [196]:
ppi_uniq_bincocomp.columns

Index(['source', 'row_number', 'idA', 'idB', 'method', 'publ', 'idtype_A',
       'idtype_B', 'UniProt_A', 'UniProt_B', 'taxa', 'gene_name_A',
       'entry_name_A', 'best_uprotA', 'gene_name_B', 'entry_name_B',
       'best_uprotB', 'ppi', 'pmid_method', 'type', 'is_binary_pub',
       'is_cocomp_pub', 'quality', 'pmid:method:quality'],
      dtype='object')

In [197]:
ppi_uniq_bincocomp['is_binary_hq'] = ppi_uniq_bincocomp.apply(lambda x: is_high_quality_binary(x, special_cases['forced_binary']), axis=1)
ppi_uniq_bincocomp['is_cocomp_hq'] = ppi_uniq_bincocomp.apply(lambda x: is_high_quality_cocomp(x, special_cases['forced_cocomp'], to_cocomp_publ), axis=1)

In [198]:
ppi_uniq_bincocomp['is_cocomp_hq'] = ppi_uniq_bincocomp['is_cocomp_hq'] & ppi_uniq_bincocomp['is_cocomp_pub']

In [202]:
# forced_cocomp_all = to_cocomp_publ + list(special_cases['forced_cocomp'])
# ppi_uniq_bincocomp['is_hq'] = ppi_uniq_bincocomp.apply(lambda x: is_high_quality(x, special_cases['forced_binary'], forced_cocomp_all), axis=1)

In [203]:
# ppi_uniq_bincocomp.groupby(['type', 'is_hq'])['ppi'].nunique()

In [204]:
# ppi_uniq_bincocomp.groupby(['type', 'quality', 'is_hq'])['ppi'].nunique()

[revision 2024.4.16] --- add interaction type to `pmid:method:quality` triplet

In [205]:
# revision 2024.4.16 --- load from cache
# dtype_dict = {'idA': str, 'idB': str, 'gene_name_A': str, 'gene_name_B': str, 'method': str, 'publ': str}
# ppi_uniq_bincocomp = pd.read_csv(output_root / 'bincocomp_interactome.txt', sep='\t', dtype=dtype_dict)

In [207]:
ppi_uniq_bincocomp.columns

Index(['source', 'row_number', 'idA', 'idB', 'method', 'publ', 'idtype_A',
       'idtype_B', 'UniProt_A', 'UniProt_B', 'taxa', 'gene_name_A',
       'entry_name_A', 'best_uprotA', 'gene_name_B', 'entry_name_B',
       'best_uprotB', 'ppi', 'pmid_method', 'type', 'is_binary_pub',
       'is_cocomp_pub', 'quality', 'pmid:method:quality', 'is_binary_hq',
       'is_cocomp_hq'],
      dtype='object')

In [206]:
ppi_uniq_bincocomp['type'].unique()

array(['binary', 'co-complex', nan], dtype=object)

In [43]:
# ppi_uniq_bincocomp[ppi_uniq_bincocomp['type'].isna()].drop_duplicates('pmid_method')['publ'].isin(special_cases['binary_blacklist_forced_cocomplex']).all()

In [208]:
# revise column name to avoid ambiguity (type --> method_type, not finalized)
ppi_uniq_bincocomp = ppi_uniq_bincocomp.rename(columns={'type': 'method_type'})

In [209]:
# sanity check
# each `pmid:method:quality` triplet should be exactly one of binary or co-complex
assert not (ppi_uniq_bincocomp['is_binary_pub'] & ppi_uniq_bincocomp['is_cocomp_pub']).any()
assert (ppi_uniq_bincocomp['is_binary_pub'] | ppi_uniq_bincocomp['is_cocomp_pub']).all()

In [46]:
ppi_uniq_bincocomp.columns

Index(['source', 'row_number', 'idA', 'idB', 'method', 'publ', 'idtype_A',
       'idtype_B', 'UniProt_A', 'UniProt_B', 'taxa', 'gene_name_A',
       'entry_name_A', 'best_uprotA', 'gene_name_B', 'entry_name_B',
       'best_uprotB', 'ppi', 'pmid_method', 'method_type', 'is_binary_pub',
       'is_cocomp_pub', 'quality', 'is_hq', 'pmid:method:quality'],
      dtype='object')

In [210]:
# ppi_type --- consider both method_type (based on evidence code) and publication
ppi_uniq_bincocomp['ppi_type'] = ppi_uniq_bincocomp.apply(lambda x: 'binary' if x['is_binary_pub'] else 'co-complex', axis=1)
ppi_uniq_bincocomp['pmid:method:quality:type'] = ppi_uniq_bincocomp['pmid:method:quality'] + ':' + ppi_uniq_bincocomp['ppi_type']

In [211]:
ppi_uniq_bincocomp['ppi_type'].value_counts()

binary        1646177
co-complex    1282179
Name: ppi_type, dtype: int64

In [54]:
ppi_uniq_bincocomp['ppi_type'].value_counts()

binary        1402267
co-complex    1286181
Name: ppi_type, dtype: int64

In [213]:
ppi_uniq_bincocomp.query('is_binary_hq == True')['ppi_type'].value_counts()

binary    763930
Name: ppi_type, dtype: int64

In [212]:
ppi_uniq_bincocomp.query('is_cocomp_hq == True')['ppi_type'].value_counts()

co-complex    403080
Name: ppi_type, dtype: int64

In [215]:
ppi_uniq_bincocomp.query('is_cocomp_hq == True & ppi_type == "binary"')['source'].unique()

array([], dtype=object)

#### Fill gene information

In [216]:
# curate uniprot-to-gene mapping from current interaction data
idx = ppi_uniq_bincocomp['gene_name_A'].notna()
uprot2gene = dict(zip(ppi_uniq_bincocomp.loc[idx, 'best_uprotA'], ppi_uniq_bincocomp.loc[idx, 'gene_name_A']))
print(len(uprot2gene))

idx = ppi_uniq_bincocomp['gene_name_B'].notna()
uprot2gene.update(dict(zip(ppi_uniq_bincocomp.loc[idx, 'best_uprotB'], ppi_uniq_bincocomp.loc[idx, 'gene_name_B'])))
print(len(uprot2gene))

94176
120162


In [244]:
ppi_uniq_bincocomp.to_csv(output_root / 'bincocomp_interactome.txt', sep='\t', index=False)

#### Aggregate by interaction

--- each row becomes a unique interaction with all publication+method+quality information concatenated

**Aggregating criteria**
* An interaction is considered as **binary** if **any** publication + evidence code suggests it is binary
* An interaction is considered as **co-complex** if **any** publication + evidence code suggests it is co-complex
* High quality if **any** single record meets **high-quality** condition **or** has >1 publications not in the blacklist

* [revision 2024.4.16] change `pmid:method:quality` into `pmid:method:quality:type` (add interaction type)
* [revision 2024.4.16] add `high_quality` column to HINT format output

In [219]:
# Load previous data
dtype_dict = {'method': str, 'idA': str, 'idB': str, 'taxa': str, 'publ': str}

ppi_uniq_bincocomp = pd.read_csv(output_root / 'bincocomp_interactome.txt', sep='\t', dtype=dtype_dict)  # binary / co-complex (keep seperate rows for different publications/methods)

In [220]:
ppi_uniq_bincocomp.query('ppi_type == "binary"')[['method_type', 'is_binary_pub']].describe()

,method_type,is_binary_pub
count,1598651,1646177
unique,1,1
top,binary,True
freq,1598651,1646177


In [221]:
# revised: count publication by interaction type
ppi2pub_bytype = {'binary': ppi_uniq_bincocomp.query('ppi_type == "binary"').groupby(['ppi']).agg({'publ': set}).to_dict()['publ'],
                  'cocomp': ppi_uniq_bincocomp.query('ppi_type == "co-complex"').groupby(['ppi']).agg({'publ': set}).to_dict()['publ']}

In [222]:
len(ppi2pub_bytype['binary'])

826440

In [241]:
len(ppi2pub_bytype['cocomp'])

845826

In [242]:
ppi_agg_bincomp = ppi_uniq_bincocomp.groupby(['ppi', 'taxa']).agg({'source': set, 'pmid:method:quality:type': set, 'publ': set, 
                                                                   'is_binary_hq': any, 'is_cocomp_hq': any, 'is_binary_pub': any, 'is_cocomp_pub': any}).reset_index()
ppi_agg_bincomp['source'] = ppi_agg_bincomp['source'].apply(lambda x: '|'.join(sorted(x)))
ppi_agg_bincomp['pmid:method:quality:type'] = ppi_agg_bincomp['pmid:method:quality:type'].apply(lambda x: '|'.join(sorted(x)))
# ppi_agg_bincomp['n_valid_pub'] = ppi_agg_bincomp['publ'].apply(len)
ppi_agg_bincomp = ppi_agg_bincomp.rename(columns={'is_binary_pub': 'is_binary', 'is_cocomp_pub': 'is_cocomp'})
# .drop('publ', axis=1)
# high quality tag when aggregating information from all publications
# ppi_agg_bincomp['is_hq_agg'] = ppi_agg_bincomp['is_hq'] | (ppi_agg_bincomp['n_valid_pub'] > 1)

In [243]:
ppi_agg_bincomp.head()

,ppi,taxa,source,pmid:method:quality:type,publ,is_binary_hq,is_cocomp_hq,is_binary,is_cocomp
0,A0A014C6J9:A0A014C6J9,1310912,PDB,29438671:0114:LC:binary,{29438671},True,False,True,False
1,A0A021WW32:A0A021WW32,7227,BioGrid,27996020:0004:LC:co-complex,{27996020},False,False,False,True
2,A0A021WW32:A1Z8S6,7227,BioGrid,25588834:0004:LC:co-complex,{25588834},False,False,False,True
3,A0A021WW32:D6W4X5,7227,IntAct,37061542:0397:LC:binary|37061542:1112:LC:binary,{37061542},False,False,True,False
4,A0A021WW32:P05205,7227,BioGrid,24990964:0004:LC:co-complex,{24990964},False,False,False,True


In [245]:
ppi_agg_bincomp['n_binary_pub'] = ppi_agg_bincomp['ppi'].apply(lambda x: len(ppi2pub_bytype['binary'].get(x, [])))
ppi_agg_bincomp['n_cocomp_pub'] = ppi_agg_bincomp['ppi'].apply(lambda x: len(ppi2pub_bytype['cocomp'].get(x, [])))

In [246]:
ppi_agg_bincomp['is_binary_hq_agg'] = ppi_agg_bincomp['is_binary_hq'] | (ppi_agg_bincomp['n_binary_pub'] > 1)
ppi_agg_bincomp['is_cocomp_hq_agg'] = ppi_agg_bincomp['is_cocomp_hq'] | (ppi_agg_bincomp['n_cocomp_pub'] > 1)

In [247]:
ppi_agg_bincomp.head()

,ppi,taxa,source,pmid:method:quality:type,publ,is_binary_hq,is_cocomp_hq,is_binary,is_cocomp,n_binary_pub,n_cocomp_pub,is_binary_hq_agg,is_cocomp_hq_agg
0,A0A014C6J9:A0A014C6J9,1310912,PDB,29438671:0114:LC:binary,{29438671},True,False,True,False,1,0,True,False
1,A0A021WW32:A0A021WW32,7227,BioGrid,27996020:0004:LC:co-complex,{27996020},False,False,False,True,0,1,False,False
2,A0A021WW32:A1Z8S6,7227,BioGrid,25588834:0004:LC:co-complex,{25588834},False,False,False,True,0,1,False,False
3,A0A021WW32:D6W4X5,7227,IntAct,37061542:0397:LC:binary|37061542:1112:LC:binary,{37061542},False,False,True,False,1,0,False,False
4,A0A021WW32:P05205,7227,BioGrid,24990964:0004:LC:co-complex,{24990964},False,False,False,True,0,1,False,False


In [248]:
# Fill back information
ppi_agg_bincomp['UniProt_A'], ppi_agg_bincomp['UniProt_B'] = zip(*ppi_agg_bincomp['ppi'].apply(lambda x: x.split(':')))
ppi_agg_bincomp['Gene_A'] = ppi_agg_bincomp['UniProt_A'].apply(lambda x: uprot2gene.get(x, np.nan))
ppi_agg_bincomp['Gene_B'] = ppi_agg_bincomp['UniProt_B'].apply(lambda x: uprot2gene.get(x, np.nan))

In [249]:
ppi_agg_bincomp.query('taxa == "9606" & is_binary == True').groupby('is_binary_hq_agg')['ppi'].nunique()

is_binary_hq_agg
False    329825
True     160024
Name: ppi, dtype: int64

In [253]:
# ppi_agg_bincomp.query('taxa == "9606" & is_binary == True').groupby('is_binary_hq_agg')['ppi'].nunique()

In [81]:
ppi_agg_bincomp.query('taxa == "9606" & is_binary == True')['ppi'].nunique()

305994

In [251]:
ppi_agg_bincomp.query('taxa == "9606" & is_cocomp == True')['ppi'].nunique()

596971

In [252]:
ppi_agg_bincomp.query('taxa == "9606" & is_cocomp == True').groupby('is_cocomp_hq_agg')['ppi'].nunique()

is_cocomp_hq_agg
False    400734
True     196237
Name: ppi, dtype: int64

In [254]:
# save to cache if needed
ppi_uniq_bincocomp.to_csv(output_root / 'bincocomp_interactome.txt', index=False, sep='\t')  # binary / co-complex (keep seperate rows for different publications/methods)
ppi_agg_bincomp.to_csv(output_root / 'bincocomp_agg_interactome.txt', index=False, sep='\t') # binary / co-complex (aggregated by interaction)

#### Generate HINT format output

In [255]:
output_root

PosixPath('/home/yl986/data/HINT/update_2024/outputs')

In [256]:
# ppi_agg_bincomp = pd.read_csv(output_root / 'bincocomp_agg_interactome.txt', sep='\t')

In [257]:
# hint_output_root = output_root / 'HINT_format'
hint_output_root = output_root / 'HINT_format'

if not hint_output_root.exists():
    hint_output_root.mkdir(parents=True)

In [258]:
ppi_agg_bincomp.head()

,ppi,taxa,source,pmid:method:quality:type,publ,is_binary_hq,is_cocomp_hq,is_binary,is_cocomp,n_binary_pub,n_cocomp_pub,is_binary_hq_agg,is_cocomp_hq_agg,UniProt_A,UniProt_B,Gene_A,Gene_B
0,A0A014C6J9:A0A014C6J9,1310912,PDB,29438671:0114:LC:binary,{29438671},True,False,True,False,1,0,True,False,A0A014C6J9,A0A014C6J9,J811_1009,J811_1009
1,A0A021WW32:A0A021WW32,7227,BioGrid,27996020:0004:LC:co-complex,{27996020},False,False,False,True,0,1,False,False,A0A021WW32,A0A021WW32,vtd,vtd
2,A0A021WW32:A1Z8S6,7227,BioGrid,25588834:0004:LC:co-complex,{25588834},False,False,False,True,0,1,False,False,A0A021WW32,A1Z8S6,vtd,pds5
3,A0A021WW32:D6W4X5,7227,IntAct,37061542:0397:LC:binary|37061542:1112:LC:binary,{37061542},False,False,True,False,1,0,False,False,A0A021WW32,D6W4X5,vtd,CG15250-RA
4,A0A021WW32:P05205,7227,BioGrid,24990964:0004:LC:co-complex,{24990964},False,False,False,True,0,1,False,False,A0A021WW32,P05205,vtd,Su(var)205


In [259]:
ppi_agg_bincomp.query('is_binary_hq_agg == True')['pmid:method:quality:type'].str.contains('binary').all()

True

In [260]:
ppi_agg_bincomp.query('is_cocomp_hq_agg == True')['pmid:method:quality:type'].str.contains('complex').all()

True

In [261]:
# HINT format with selected columns
cols = ["Uniprot_A", "Uniprot_B", "Gene_A", "Gene_B", "pmid:method:quality:type", "taxid", "high_quality_binary", 'high_quality_cocomp']

ppi_output = ppi_agg_bincomp.rename(columns={'UniProt_A': 'Uniprot_A', 'UniProt_B': 'Uniprot_B', 'taxa': 'taxid', 
                                             'is_binary_hq_agg': 'high_quality_binary', 
                                             'is_cocomp_hq_agg': 'high_quality_cocomp'}).sort_values(cols)

In [262]:
ppi_output.head()[cols]

,Uniprot_A,Uniprot_B,Gene_A,Gene_B,pmid:method:quality:type,taxid,high_quality_binary,high_quality_cocomp
0,A0A014C6J9,A0A014C6J9,J811_1009,J811_1009,29438671:0114:LC:binary,1310912,True,False
1,A0A021WW32,A0A021WW32,vtd,vtd,27996020:0004:LC:co-complex,7227,False,False
2,A0A021WW32,A1Z8S6,vtd,pds5,25588834:0004:LC:co-complex,7227,False,False
3,A0A021WW32,D6W4X5,vtd,CG15250-RA,37061542:0397:LC:binary|37061542:1112:LC:binary,7227,False,False
4,A0A021WW32,P05205,vtd,Su(var)205,24990964:0004:LC:co-complex,7227,False,False


**[Update 2024-06-01]: fix bug for high-quality filtering**

In [263]:
ppi_output.query('is_binary == True & is_cocomp == False  & high_quality_cocomp')

,ppi,taxid,source,pmid:method:quality:type,publ,is_binary_hq,is_cocomp_hq,is_binary,is_cocomp,n_binary_pub,n_cocomp_pub,high_quality_binary,high_quality_cocomp,Uniprot_A,Uniprot_B,Gene_A,Gene_B


In [264]:
ppi_output.query('high_quality_binary == True | high_quality_cocomp == True')[cols + ['is_binary', 'is_cocomp']].describe()

,Uniprot_A,Uniprot_B,Gene_A,Gene_B,pmid:method:quality:type,taxid,high_quality_binary,high_quality_cocomp,is_binary,is_cocomp
count,575588,575588,570423,571231,575588,575588,575588,575588,575588,575588
unique,65312,68264,50106,52716,81165,3654,2,2,2,2
top,P11484,Q69YN4,SSB1,VIRMA,32296183:0018:HT:binary|32296183:0397:HT:binar...,9606,True,True,True,True
freq,3301,1474,3302,1476,29373,336217,306100,299103,394008,310777


In [265]:
def update_evidence(ev_string, keyword, sep='|'):
    parts = ev_string.split('|')
    udpate_parts = []
    for p in parts:
        if keyword in p:
            udpate_parts.append(p)
    return '|'.join(udpate_parts)

In [266]:
ppi_output_binary = ppi_output.query('is_binary == True')[cols].drop('high_quality_cocomp', axis=1).rename(columns={'high_quality_binary': 'high_quality'}).reset_index(drop=True)
ppi_output_binary['pmid:method:quality:type'] = ppi_output_binary['pmid:method:quality:type'].apply(lambda x: update_evidence(x, 'binary'))

ppi_output_cocomp = ppi_output.query('is_cocomp == True')[cols].drop('high_quality_binary', axis=1).rename(columns={'high_quality_cocomp': 'high_quality'}).reset_index(drop=True)
ppi_output_cocomp['pmid:method:quality:type'] = ppi_output_cocomp['pmid:method:quality:type'].apply(lambda x: update_evidence(x, 'co-complex'))

In [268]:
ppi_output[cols].to_csv(hint_output_root / 'both_all.txt', index=False, sep='\t')
# ppi_output.query('is_binary == True')[cols].to_csv(hint_output_root / 'binary_all.txt', index=False, sep='\t')
# ppi_output.query('is_cocomp == True')[cols].to_csv(hint_output_root / 'cocomp_all.txt', index=False, sep='\t')
ppi_output_binary.to_csv(hint_output_root / 'binary_all.txt', index=False, sep='\t')
ppi_output_cocomp.to_csv(hint_output_root / 'cocomp_all.txt', index=False, sep='\t')
# ppi_output.query('high_quality == True')[cols].to_csv(hint_output_root / 'both_hq.txt', index=False, sep='\t')

In [269]:
ppi_output.query('high_quality_binary == True | high_quality_cocomp == True')[cols].to_csv(hint_output_root / 'both_hq.txt', index=False, sep='\t')

In [270]:
ppi_output_binary.query('high_quality == True').to_csv(hint_output_root / 'binary_hq.txt', index=False, sep='\t')
ppi_output_cocomp.query('high_quality == True').to_csv(hint_output_root / 'cocomp_hq.txt', index=False, sep='\t')

#### HINT format interactome by species

* Load species supporting documents

In [271]:
df_species = pd.read_csv(source_root / 'docs/cache/species_parsed.txt', sep='\t')
with open(output_root / 'cache/taxa_map_dict.json', 'r') as f:
    taxa_map_dict = json.load(f)

In [272]:
# Rule-based curation (consider to revise later)
df_species['scientific_name_short'] = df_species['scientific_name'].apply(lambda x: x.split('(')[0].strip())
ecoli_idx = df_species['scientific_name'].str.startswith('Escherichia coli')  # manual rule for E Coli
df_species.loc[ecoli_idx, 'scientific_name_short'] = 'Escherichia coli'

rice_idx = df_species['common_name'] == 'Rice' # manual rule for Rice
df_species.loc[rice_idx, 'scientific_name_short'] = 'Oryza sativa'

In [273]:
df_species['taxa'] = df_species['taxa'].astype(str)
df_species[['taxa', 'scientific_name_short']].drop_duplicates().sort_values('taxa').to_csv(output_root / 'taxid2name_short.txt', index=False, sep='\t')

* Target species

Create separated directory for each species using species name (Pascal case naming rule, remove space and capitalize the first letter for each word)

In [274]:
target_species = ['Homo sapiens',
                  'Saccharomyces cerevisiae',
                  'Caenorhabditis elegans',
                  'Arabidopsis thaliana',
                  'Mus musculus',
                  'Escherichia coli',
                  'Rattus norvegicus',
                  'Oryza sativa',
                  'Schizosaccharomyces pombe',
                  'Drosophila melanogaster']
target_species = [s.lower() for s in target_species]

In [275]:
df_spe_target = df_species[df_species['scientific_name_short'].str.lower().isin(target_species)].reset_index(drop=True)
df_spe_target = df_spe_target.groupby('scientific_name_short').agg({'taxa': list}).reset_index()
select_name2id = dict(zip(df_spe_target['scientific_name_short'], df_spe_target['taxa']))

In [276]:
with open(output_root / 'select_species2id.json', 'w') as f:
    json.dump(select_name2id, f, indent=2)

In [277]:
flist = list(hint_output_root.glob('*txt'))

for fpath in flist:
    if fpath.name.startswith('protein_meta'):  # skip protein description file
        continue
    df = pd.read_csv(fpath, sep='\t', dtype=str)
    suffix = fpath.name
    
    for species, taxa_list in select_name2id.items():
        species_tag = re.sub(' ', '', species.title()) 
        out_dir = hint_output_root / 'taxa' / species_tag
        if not(out_dir).exists():
            out_dir.mkdir(parents=True)
        out_fpath = out_dir / f'{species_tag}_{suffix}'
        
        df_cur = df[df['taxid'].isin(taxa_list)]
        if len(df_cur):
            df_cur.to_csv(out_fpath, sep='\t', index=False)
        

### Generate protein meta data

Goal: fetch protein name and description that match UniProt IDs in each interaction 

(in curated interactome data, some protein name & description don't match the displayed UniProt ID because we selected the **best** UniProt ID by gene)

In [278]:
to_best_uprot = pd.read_csv(output_root / 'cache/id_to_best_uprot_uniq.txt', sep='\t', dtype={'id': str, 'taxa': str})
prot_desc = pd.read_csv(output_root / 'cache/uniprot_descriptions.txt', sep='\t', dtype=str)

In [279]:
meta_cols = ['best_uprot_full', 'taxa', 'gene_name_fil']
prot_meta = to_best_uprot[meta_cols].rename(columns={'best_uprot_full': 'uniprot', 'gene_name_fil': 'gene'}).astype({'taxa': str}).drop_duplicates().reset_index(drop=True)
prot_meta['primary_short'] = prot_meta['uniprot'].apply(lambda x: x.split('-')[0])

In [280]:
# Load taxa ID supporting document
with open(output_root / 'cache/taxa_map_dict.json') as f:
    taxa_map_dict = json.load(f)

In [281]:
# curate taxonomy ID
prot_desc['taxa'] = prot_desc['taxa'].astype(str).apply(lambda x: taxa_map_dict.get(x, str(x)))

In [282]:
# Merge information
prot_meta_complete = prot_meta.merge(prot_desc[['UniProt', 'name', 'taxa', 'description', 'tag']].rename(columns={'UniProt': 'primary_short'}))
prot_meta_complete['is_reviewed'] = (prot_meta_complete['tag'] == 'sp')

In [283]:
# save protein meta to file
prot_meta_complete.sort_values(['uniprot']).drop('tag', axis=1).to_csv(output_root / 'HINT_format/protein_meta.txt', index=False, sep='\t')

In [ ]:
len(special_cases['binary_blacklist_forced_cocomplex'])

27

In [ ]:
special_cases['binary_blacklist_forced_cocomplex']

{'23667531',
 '25959826',
 '26686862',
 '26739451',
 '26949739',
 '28514442',
 '29476059',
 '29858488',
 '29961565',
 '30091672',
 '30097533',
 '30300385',
 '30698747',
 '30886144',
 '31046837',
 '31308550',
 '31388001',
 '31391242',
 '31402609',
 '31413325',
 '31536960',
 '31616248',
 '31980649',
 '32357304',
 '32552912',
 '32774160',
 '32900848'}